# Word Embeddings : le modèle Word2Vec

## Imports

In [1]:
import sys

from gensim.models.phrases import Phrases, Phraser
from gensim.models import Word2Vec

import nltk
from nltk.tokenize import wordpunct_tokenize
from unidecode import unidecode

## Chargement et traitement des phrases du corpus

### Création d'un objet qui *streame* les lignes d'un fichier pour économiser de la RAM

In [2]:
class MySentences(object):
    """Tokenize and Lemmatize sentences"""
    def __init__(self, filename):
        self.filename = filename

    def __iter__(self):
        for line in open(self.filename, encoding='utf-8', errors="backslashreplace"):
            yield [unidecode(w.lower()) for w in wordpunct_tokenize(line)]

In [3]:
infile = f"../data/sents.txt"
sentences = MySentences(infile)

### Détection des bigrams

Article intéressant sur le sujet : https://towardsdatascience.com/word2vec-for-phrases-learning-embeddings-for-more-than-one-word-727b6cf723cf

In [4]:
bigram_phrases = Phrases(sentences)

L'object `phrases` peut être vu comme un large dictionnaire d'expressions multi-mots associées à un score, le *PMI-like scoring*. Ce dictionnaire est construit par un apprentissage sur base d'exemples.
Voir les références ci-dessous :
- https://arxiv.org/abs/1310.4546
- https://en.wikipedia.org/wiki/Pointwise_mutual_information

In [5]:
type(bigram_phrases.vocab)

dict

Il contient de nombreuses clés qui sont autant de termes observés dans le corpus

In [6]:
len(bigram_phrases.vocab.keys())

15850647

Prenons une clé au hasard :

In [7]:
key_ = list(bigram_phrases.vocab.keys())[144]
print(key_)

1q


Le dictionnaire indique le score de cette coocurrence :

In [8]:
bigram_phrases.vocab[key_]

488

Lorsque l'instance de `Phrases` a été entraînée, elle peut concaténer les bigrams dans les phrases lorsque c'est pertinent.

### Conversion des `Phrases` en objet `Phraser`

`Phraser` est un alias pour `gensim.models.phrases.FrozenPhrases`, voir ici https://radimrehurek.com/gensim/models/phrases.html.

Le `Phraser` est une version *light* du `Phrases`, plus optimale pour transformer les phrases en concaténant les bigrams.

In [9]:
bigram_phraser = Phraser(phrases_model=bigram_phrases)

Le `Phraser` est un objet qui convertit certains unigrams d'une liste en bigrams lorsqu'ils ont été identifiés comme pertinents.

### Extraction des trigrams

Nous répétons l'opération en envoyant cette fois la liste de bigrams afin d'extraire les trigrams.

In [10]:
trigram_phrases = Phrases(bigram_phraser[sentences])

In [11]:
trigram_phraser = Phraser(phrases_model=trigram_phrases)

### Création d'un corpus d'unigrams, bigrams, trigrams

In [12]:
corpus = list(trigram_phraser[bigram_phraser[sentences]])

In [13]:
print(corpus[:100])

[['mi', 'imnri', 'r', 'i', '<<', 'i', 'i', 'hmu', "'", 'i', '/', 'tx', "-'", 'l', ':', 'marche', 'tenu', 'hors', 'villa', ',', 'la', '9', '.'], ['--', 'u', 'a', 'ete', 'vaain', 'si', 'teicj', '>>', 'm', 'races_indigenes', 'de', 'fr', '.'], ['31', '<)', 'a', '5s', "'", 'k', '131', 'de', '.'], ['rasa', 'iichakdui', "'", 'te', ',', 'do', '(', 'r', '.', '3s0', 'h', '710', '.', 'taureaux', 'iallsenas', ',>', 'ia', 'u', '\\', '--', 'a', '--', ';', '0ii', '.'], ['hollandais', ',', 'dufr', '.'], ['0', '.'], ['--', 'a', '9', '.--', 'la', 'idto', '-', 'vachei', 'laitieres', ':', 'bn', 'vante', '1q', '.'], ['vendues', '3', '\\', 'au', 'prix', 'la', '410', 'a', '*', '<<', 'i', 'h', '\\;', 'genisses', ',', 'kl', '.'], ["'.", '9', '.'], ['i', 'l', '.', '2', 'i', '.', 'id', '.'], ['da', '370', 'i', '6lutr', '.'], ['marche', 'a', '<', 'u', 'porcs', '.'], ['--', 'categorie', 'de', 'lt', 'ilashtya', ':', "'", '237', 'on', 'vente', ';', 'vendus', '1', 'm', '.', 'do', "'", '2', 'i', '.--', 'a', ';:,', 'l'

## Entrainement d'un modèle Word2Vec sur ce corpus

In [14]:
%%time
model = Word2Vec(
    corpus, # On passe le corpus de ngrams que nous venons de créer
    vector_size=100, # Le nombre (32) de dimensions dans lesquelles le contexte des mots devra être réduit, aka. vector_size
    window=1000, # La taille du "contexte", ici 5 mots avant et après le mot observé
    min_count=5, # On ignore les mots qui n'apparaissent pas au moins 5 fois dans le corpus
    workers=4, # Permet de paralléliser l'entraînement du modèle en 4 threads
    epochs=5 # Nombre d'itérations du réseau de neurones sur le jeu de données pour ajuster les paramètres avec la descente de gradient, aka. epochs.
)

CPU times: user 30min 20s, sys: 1min 50s, total: 32min 11s
Wall time: 10min 43s


#### Remarque

Vous voyez ici que l'entrainement du modèle est parallélisé (sur 4 workers).

Lors qu'on parallélise l'entrainement du modèle, 4 modèles "séparés" sont entrainés sur environ un quart des phrases.

Ensuite, les résultats sont agrégés pour ne plus faire qu'un seul modèle.

On ne peut prédire quel worker aura quelle phrase, car il y a des aléas lors de la parallélisation (p. ex. un worker qui serait plus lent, etc.).

Du coup, les valeurs peuvent varier légèrement d'un entrainement à l'autre.

Mais, globalement, les résultats restent cohérents.

### Sauver le modèle dans un fichier

In [15]:
outfile = f"../data/newspapers.model"
model.save(outfile)

## Explorer le modèle

### Charger le modèle en mémoire

In [16]:
model = Word2Vec.load("../data/newspapers.model")

### Imprimer le vecteur d'un terme

In [17]:
model.wv["ministre"]

array([  0.3923662 ,  -3.1683297 ,   2.1926095 ,   3.4758174 ,
         0.088103  ,  -6.0576744 ,   7.2843604 ,  -2.066003  ,
        -0.26695386, -10.8539295 ,  -2.9728537 ,  -0.3956999 ,
         6.7759995 ,   7.0304165 ,   5.071722  ,   1.9317945 ,
         5.2799735 ,   4.0125027 ,  -5.173631  ,  -0.5705354 ,
         3.5173378 ,  -1.1700752 ,  -9.176954  ,   3.4244251 ,
        -4.6593738 ,   3.0211096 ,   0.7595081 ,   3.6076102 ,
         4.1991053 ,  -1.1660833 ,  -1.1378769 ,  -3.5877898 ,
         6.586602  ,   2.7594337 ,  -2.0445786 ,   1.0987376 ,
        -4.0814123 ,  11.948199  ,   4.1576514 ,   3.8055122 ,
        -8.624517  ,   4.537557  ,  11.514919  ,   4.477151  ,
        -7.4238906 ,   1.698307  ,  -7.471587  ,  -6.8405385 ,
         1.091742  ,  -6.9448295 ,  -5.189336  ,   2.1725569 ,
        -4.385874  ,   3.3794708 ,  -2.374395  ,  -3.1021914 ,
        -5.3150234 ,  -5.2095957 ,   1.5176932 ,   8.4552145 ,
         5.904068  ,  -6.875185  ,   1.6016647 ,   0.43

### Calculer la similarité entre deux termes

In [18]:
model.wv.similarity("pere", "mere")

0.7286005

In [19]:
model.wv.similarity("le", "un")

0.81265134

In [20]:
model.wv.similarity("football", "hockey")

0.86579925

### Chercher les mots les plus proches d'un terme donné

In [28]:
model.wv.most_similar("femme", topn=20)

[('femmo', 0.8186554312705994),
 ('jeune_femme', 0.7793405652046204),
 ('petite_fille', 0.6991528272628784),
 ('nourrice', 0.6868636608123779),
 ('fille', 0.6784239411354065),
 ('jeune_fille', 0.6720597147941589),
 ('fillette', 0.6530797481536865),
 ('femme_mariee', 0.6486173272132874),
 ('mariee', 0.6439478993415833),
 ('iemme', 0.6352880001068115),
 ('jeune_tille', 0.6266009211540222),
 ('sorciere', 0.6211362481117249),
 ('vieille_dame', 0.6209268569946289),
 ('blanchisseuse', 0.6077374219894409),
 ('mari', 0.6061207056045532),
 ('fomme', 0.6041224002838135),
 ('maitresse', 0.6028937697410583),
 ('jeune_veuve', 0.6013625264167786),
 ('vieille_femme', 0.6001562476158142),
 ('servante', 0.5996935963630676)]

In [22]:
model.wv.most_similar("hiver", topn=20)

[('automne', 0.8073281049728394),
 ('saison', 0.648826003074646),
 ('printemps', 0.6427499651908875),
 ('estivale', 0.5682312250137329),
 ('hlver', 0.561298131942749),
 ('cet_hiver', 0.5388601422309875),
 ('pluie', 0.5339532494544983),
 ('paques', 0.5328870415687561),
 ('pentecote', 0.5296821594238281),
 ('neige', 0.5287347435951233),
 ('pluies', 0.5274837613105774),
 ('semis', 0.5254013538360596),
 ('oiseaux', 0.5241944193840027),
 ('notre_climat', 0.5227209329605103),
 ('chaud', 0.5202927589416504),
 ('froids', 0.5139855742454529),
 ('fraiches', 0.5139128565788269),
 ('ardenne', 0.5128042101860046),
 ('abondance', 0.5096802711486816),
 ('hiver_prochain', 0.508560299873352)]

In [23]:
model.wv.most_similar("balkans", topn=20)

[('soviets', 0.7569776177406311),
 ('frontieres', 0.7271795868873596),
 ('troupes_sovietiques', 0.7250498533248901),
 ('operations_militaires', 0.7160229682922363),
 ('armees', 0.7060896158218384),
 ('troupes_allemandes', 0.7001009583473206),
 ('grandes_puissances', 0.699258029460907),
 ('puissances', 0.6980260610580444),
 ('troupes_russes', 0.6936063766479492),
 ('occidentale', 0.6888675689697266),
 ('pays_arabes', 0.6879344582557678),
 ('pactes', 0.6878247261047363),
 ('russes', 0.6837083101272583),
 ('turquie', 0.6826925277709961),
 ('balkaniques', 0.682327151298523),
 ('serbes', 0.6809642910957336),
 ('fronts', 0.6752739548683167),
 ('occidentales', 0.6749254465103149),
 ('bulgares', 0.6704584956169128),
 ('forces_allemandes', 0.6701508164405823)]

### Faire des recherches complexes à travers l'espace vectoriel

In [24]:
print(model.wv.most_similar(positive=['uruguay', 'chine'], negative=['europe']))

[('bolivie', 0.6309815645217896), ('bresil', 0.6149224042892456), ('chine_1921', 0.6090151071548462), ('minas', 0.5764032006263733), ('hong', 0.5751096606254578), ('chili', 0.5711575150489807), ('chine_1903', 0.5707996487617493), ('perou', 0.564791738986969), ('argentine', 0.5646613836288452), ('bresil_1889', 0.5576472282409668)]


In [25]:
print(model.wv.most_similar(positive=['esturgeon', 'hareng'], negative=['legumes']))

[('cresson_compote', 0.6932607889175415), ('huitres_royales', 0.6882269978523254), ('sarcelle', 0.6838253140449524), ('meim', 0.6829068660736084), ('francs_huitres', 0.673295259475708), ('sauce_mousseline', 0.6712167859077454), ('pommes_nouvelles', 0.6691917181015015), ('rhone_rotie', 0.6660453677177429), ('poularde', 0.6649585962295532), ('meuniere', 0.6598445177078247)]


In [26]:
print(model.wv.most_similar(positive=['poumons', 'foie'], negative=['maladie']))

[('purifie', 0.7186546325683594), ('impuretes', 0.7088902592658997), ('dissout', 0.7086023688316345), ('sirop', 0.7033413052558899), ('pores', 0.7007634043693542), ('gouttes', 0.6869702935218811), ('ongles', 0.6807660460472107), ('toniques', 0.6793760061264038), ('sels', 0.6775674819946289), ('poisons', 0.6733810305595398)]
